## 1. 환경 설정

### 1-1. 환경 변수

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### 1-2. 기본 라이브러리

In [2]:
import re
import json

from textwrap import dedent
from pprint import pprint

import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import uuid

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 30)

## 2. 데이터 설정

타겟 자산명 (직접 작성)

In [3]:
# asset_names = ['AA-ETF_EQUITY-SPY']
# asset_names = ['AA-ETF_EQUITY-XLK', 'AA-ETF_EQUITY-XLE', 'AA-ETF_EQUITY-XLF', 'AA-ETF_EQUITY-XLV', 'AA-ETF_EQUITY-XLP', 'AA-ETF_EQUITY-XLI', 'AA-ETF_EQUITY-XLU']
# asset_names = ['AA-ETF_EQUITY-EWY', 'AA-ETF_EQUITY-VWO', 'AA-ETF_EQUITY-EWC', 'AA-ETF_EQUITY-EFA', 'AA-ETF_EQUITY-EWJ', 'AA-ETF_EQUITY-EPP']
asset_names = ['AA-ETF_EQUITY-ACWV', 'AA-ETF_EQUITY-SUSL', 'AA-ETF_EQUITY-SCHD', 'AA-ETF_EQUITY-VYMI', 'AA-ETF_EQUITY-VTV'] # ['AA-ETF_EQUITY-ACWV.K', 'AA-ETF_EQUITY-SUSL.O', 'AA-ETF_EQUITY-SCHD.K', 'AA-ETF_EQUITY-VYMI.O', 'AA-ETF_EQUITY-VTV']

# cluster_name = 'SPY'
# cluster_name = 'US-SECTOR'
# cluster_name = 'exUS'
cluster_name = 'US-STYLE'

### 2-1. 자산배분

In [4]:
df_alloc = pd.read_csv('data/alloc_basdt_241120.csv')
df_alloc

,BAS_DT,ASET_GRP,ASET_GRP_WGT
0,2024-10-23,AA-ETF_BOND-BND,0.0151
1,2024-10-23,AA-ETF_BOND-BNDX,0.0210
2,2024-10-23,AA-ETF_EQUITY-ACWV,0.0397
3,2024-10-23,AA-ETF_EQUITY-EFA,0.0132
4,2024-10-23,AA-ETF_EQUITY-EPP,0.0132
...,...,...,...
478,2024-11-20,AA-ETF_EQUITY-XLK,0.2156
479,2024-11-20,AA-ETF_EQUITY-XLP,0.0025
480,2024-11-20,AA-ETF_EQUITY-XLU,0.0059
481,2024-11-20,AA-ETF_EQUITY-XLV,0.0002


### 2-2. 설명력

In [5]:
# 데이터 전처리
df_infc = pd.read_csv('data/infc_basdt_241120.csv')
df_infc['ASET_GRP'] = df_infc['ASET_LEVEL'].str[3:]
df_infc['ASET_LEVEL'] = df_infc['ASET_LEVEL'].str[:2]
df_infc = df_infc[df_infc['ASET_GRP'].isin(asset_names)] # RIC 중복 존재 (기간 통합, L1-L3)

df_infc = df_infc.merge(df_alloc, on=['BAS_DT', 'ASET_GRP'], how='inner')
df_infc = df_infc[['BAS_DT', 'ASET_GRP', 'ASET_GRP_WGT', 'RIC', 'VAR_NM', 'INFC_DIR']]
df_infc = df_infc.reset_index(drop=True)

# 테이블명 추가
df_var_map = pd.read_csv('data/var_table_mapping.csv')
df_var_map = df_var_map[['RIC', 'TABLE']]
df_var_map = df_var_map.drop_duplicates('RIC') # RIC 중복 제거 (필드만 다른 경우 존재)
df_infc = df_infc.merge(df_var_map, on='RIC', how='inner')

df_infc = df_infc.reset_index(drop=True)
df_infc # 최종

,BAS_DT,ASET_GRP,ASET_GRP_WGT,RIC,VAR_NM,INFC_DIR,TABLE
0,2024-10-23,AA-ETF_EQUITY-ACWV,0.0397,.GVZ,CBOE GOLD VOLATILITY INDEX,-,DSC101TH
1,2024-10-23,AA-ETF_EQUITY-ACWV,0.0397,.OVX,CBOE CRUDE OIL VOLATILITY INDEX,+,DSC101TH
2,2024-10-23,AA-ETF_EQUITY-ACWV,0.0397,.VXEWZ,CBOE BRAZIL ETF VOLATILITY INDEX,-,DSC101TH
3,2024-10-23,AA-ETF_EQUITY-ACWV,0.0397,CNY=,Chinese Yuan to United States Dollar (Refinitiv),-,DSC104TH
4,2024-10-23,AA-ETF_EQUITY-ACWV,0.0397,.JNIV,NIKKEI STOCK AVERAGE VOLATILITY INDEX,-,DSC101TH
...,...,...,...,...,...,...,...
976,2024-11-20,AA-ETF_EQUITY-VYMI,0.0044,CNY=,Chinese Yuan to United States Dollar (Refinitiv),-,DSC104TH
977,2024-11-20,AA-ETF_EQUITY-VYMI,0.0044,.N225,NIKKEI 225 STOCK AVERAGE,+,DSC101TH
978,2024-11-20,AA-ETF_EQUITY-VYMI,0.0044,US3YT=RRPS,Refinitiv United States Government Benchmark B...,+,DSC102TH
979,2024-11-20,AA-ETF_EQUITY-VYMI,0.0044,.INX,CME-Standard and Poors 500 Index Composite CS02,+,DSC101TH


### 2-3. 기초 데이터

In [6]:
# 전체 RIC (데이터 존재하는 것들만)
RIC_101 = ['.AORD', '.BVSP', '.CSI300', '.FTSE', '.GDAXI', '.GVZ', '.INX', '.JNIV', '.KS11', '.KSVKOSPI', '.MOVE', '.N225', '.OVX', '.SKEWX', '.STOXX50E', '.V1XI', '.V2TX', '.VHSI', '.VIX', '.VXEWZ']
RIC_102 = ['CN10YT=RR', 'CN3YT=RR', 'DE3YT=RR', 'KR3YT=RR', 'US10YT=RRPS', 'US3YT=RRPS']
RIC_103 = ['Cc1', 'FCc1', 'GCc1', 'HGc1', 'HOc1', 'JNIc1', 'KCc1', 'LCOc1', 'LHc1', 'PLc1', 'Sc1']
RIC_104 = ['BRL=', 'CNY=', 'EUR2M=', 'EUR=', 'GBP=', 'HKD=', 'JPY2M=', 'JPY=', 'KRW1M=', 'KRW2M=', 'KRW=']
RIC_105 = ['MCU0', 'XAU=']

In [7]:
# 전체 변수 (데이터 존재하는 것들만)
var_101 = [df_infc[df_infc['RIC'] == RIC_101[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_101))]
var_102 = [df_infc[df_infc['RIC'] == RIC_102[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_102))]
var_103 = [df_infc[df_infc['RIC'] == RIC_103[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_103))]
var_104 = [df_infc[df_infc['RIC'] == RIC_104[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_104))]
var_105 = [df_infc[df_infc['RIC'] == RIC_105[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_105))]

In [8]:
var_dict = {
    'Equity and Volatility Indicators': var_101,
    'Government Bond Yields': var_102,
    'Commodities and Futures': var_103,
    'Currency Exchange Rates': var_104,
    'Market Index': var_105
    }
var_dict

{'Equity and Volatility Indicators': ['Standard and Poors / ASX All Ordinaries Gold Open',
  'BRAZIL BOVESPA',
  'SHANGHAI SHENZHEN CSI 300',
  'FTSE 100',
  'DAX PERFORMANCE (XETRA)',
  'CBOE GOLD VOLATILITY INDEX',
  'CME-Standard and Poors 500 Index Composite CS02',
  'NIKKEI STOCK AVERAGE VOLATILITY INDEX',
  '[volume] Korea Stock Exchange Composite (KOSPI)',
  'VKOSPI VOLATILITY INDEX',
  'ML MOVE 1M BOND VOLATILITY INDEX',
  'NIKKEI 225 STOCK AVERAGE',
  'CBOE CRUDE OIL VOLATILITY INDEX',
  'CBOE SKEW INDEX',
  '[volume] EURO STOXX 50',
  'VDAX-NEW VOLATILITY INDEX',
  'VSTOXX VOLATILITY INDEX',
  'HSI VOLATILITY INDEX',
  'CBOE SPX VOLATILITY VIX (NEW)',
  'CBOE BRAZIL ETF VOLATILITY INDEX'],
 'Government Bond Yields': ['Refinitiv China Government Benchmark Bid Yield 10 Years',
  'Refinitiv China Government Benchmark Bid Yield 3 Years',
  'Refinitiv Germany Government Benchmark Bid Yield 3 Years',
  'Refinitiv S Korea Government Benchmark Bid Yield 3 Years',
  'Refinitiv United 

In [9]:
var_dict_101 = dict(zip(RIC_101, var_101))
var_dict_102 = dict(zip(RIC_102, var_102))
var_dict_103 = dict(zip(RIC_103, var_103))
var_dict_104 = dict(zip(RIC_104, var_104))
var_dict_105 = dict(zip(RIC_105, var_105))

In [10]:
try:
   df_101 = pd.read_csv(f'data/{cluster_name}/df_101_basdt_241120.csv')
   df_101['VAR_NM'] = df_101['RIC'].map(var_dict_101)
   df_101 = df_101[['TRADEDATE', 'RIC', 'VAR_NM', 'PI']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_101.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "PI"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_101_data = json.dumps(grouped_data, indent=2)
except:
   table_101_data = ''

In [11]:
try:
   df_102 = pd.read_csv(f'data/{cluster_name}/df_102_basdt_241120.csv')
   df_102['VAR_NM'] = df_102['RIC'].map(var_dict_102)
   df_102 = df_102[['TRADEDATE', 'RIC', 'VAR_NM', 'RY']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_102.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "RY"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_102_data = json.dumps(grouped_data, indent=2)
except:
   table_102_data = ''

In [12]:
try:
   df_103 = pd.read_csv(f'data/{cluster_name}/df_103_basdt_241120.csv')
   df_103['VAR_NM'] = df_103['RIC'].map(var_dict_103)
   df_103 = df_103[['TRADEDATE', 'RIC', 'VAR_NM', 'PS']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_103.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "PS"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_103_data = json.dumps(grouped_data, indent=2)
except:
   table_103_data = ''

In [13]:
try:
   df_104 = pd.read_csv(f'data/{cluster_name}/df_104_basdt_241120.csv')
   df_104['VAR_NM'] = df_104['RIC'].map(var_dict_104)
   df_104 = df_104[['TRADEDATE', 'RIC', 'VAR_NM', 'ER']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_104.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "ER"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_104_data = json.dumps(grouped_data, indent=2)
except:
   table_104_data = ''

In [14]:
try:
   df_105 = pd.read_csv(f'data/{cluster_name}/df_105_basdt_241120.csv')
   df_105['VAR_NM'] = df_105['RIC'].map(var_dict_105)
   df_105 = df_105[['TRADEDATE', 'RIC', 'VAR_NM', 'P']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_105.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "P"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_105_data = json.dumps(grouped_data, indent=2)
except:
   table_105_data = ''

### 2-4. 모델 설정

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

llm = ChatOpenAI(model="gpt-4o", temperature=0)

### 2-5. 설명 타겟

In [16]:
AI_model_result = df_infc.drop_duplicates(['BAS_DT', 'ASET_GRP'])[['BAS_DT', 'ASET_GRP', 'ASET_GRP_WGT']].reset_index(drop=True)

# Grouping and restructuring the data
grouped_data = []
for aset_grp, group in AI_model_result.groupby("ASET_GRP"):
    entries = group[["BAS_DT", "ASET_GRP_WGT"]].to_dict(orient="records")
    grouped_data.append({"ASET_GRP": aset_grp, "entries": entries})

# Output JSON
AI_model_result = json.dumps(grouped_data, indent=2)

AI_model_result = llm.invoke(
        [SystemMessage(content="Extract 3 things that stand out in the portfolio optimization results below. List them in order of importance. There should be no overlap.")] + 
        [HumanMessage(content=AI_model_result)]
        )
AI_model_result = AI_model_result.content

## 3. Agent

### 3-1. Analyst 생성

In [17]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

class Analyst(BaseModel):
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst.")
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    description: str = Field(description="Description of the analyst's focus, concerns, and motives.")

    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

In [18]:
import operator
from typing import List, Annotated
from typing_extensions import TypedDict

class ResearchGraphState(TypedDict):
    human_analyst_feedback: str
    analysts: List[Analyst]
    sections: Annotated[list, operator.add]
    interpretations: Annotated[list, operator.add]
    content: str

In [19]:
def create_analysts(state: ResearchGraphState):
    """ No-op node for creating analysts """
    pass

In [20]:
def human_feedback(state: ResearchGraphState):
    """ No-op node that should be interrupted on """
    pass

### 3-2. Interview 서브그래프 정의

#### A. Question 생성

In [21]:
import operator
from typing import Annotated
from langgraph.graph import MessagesState

class InterviewState(MessagesState):
    max_num_turns: int
    analyst: Analyst
    context: Annotated[list, operator.add]
    interview: str
    sections: list

In [22]:
num_subquestions = 3
max_num_turns = 3

analysts = [Analyst(name='Emma Thompson', role='Global Market Strategist', affiliation='International Financial Advisory Group', description='Emma focuses on global market dynamics, analyzing how equity and volatility indicators interact with government bond yields to influence portfolio optimization. She is particularly interested in how shifts in major indices like the DAX and FTSE 100, combined with bond yield movements, can signal broader economic trends and impact asset allocation strategies.'),
 Analyst(name='Liam Chen', role='Commodities Analyst', affiliation='Global Commodities Research Institute', description='Liam specializes in the commodities and futures markets, examining how fluctuations in key commodities like gold, copper, and crude oil affect portfolio performance. He explores the interplay between commodity prices and equity markets, assessing how these relationships can inform risk management and diversification strategies.'),
 Analyst(name='Sophia Martinez', role='Fixed Income Specialist', affiliation='Bond Market Insights', description="Sophia's expertise lies in government bond yields and their implications for portfolio optimization. She analyzes how changes in yields across different countries, such as the US, China, and Germany, influence investment decisions and risk assessments. Her focus is on understanding the macroeconomic factors driving yield movements and their impact on equity markets."),
 Analyst(name='Noah Patel', role='Currency Market Analyst', affiliation='Forex and Macro Trends', description='Noah examines the influence of currency exchange rates on global investment strategies, focusing on how currency fluctuations can affect international portfolio returns. He provides insights into how exchange rate movements, particularly involving the USD, Euro, and emerging market currencies, can impact asset performance and investor sentiment.'),
 Analyst(name='Ava Johnson', role='Volatility and Risk Analyst', affiliation='Risk Management Solutions', description='Ava is dedicated to understanding market volatility and its implications for portfolio risk management. She analyzes volatility indices, such as the VIX and VDAX, to assess market sentiment and potential risks. Her work involves developing strategies to mitigate volatility-related risks and enhance portfolio resilience in uncertain market conditions.')]

In [23]:
question_instructions = """You are an analyst tasked with interviewing an expert. 

Your goal is to explain the AI model result of portfolio optimization from your professional perspective and data.

It is extremely important to focus on gathering information on high-level themes such as market dynamics, fund flows, and potential risks.
(Caution: Focus on a subset of explanatory variables. DO NOT focus on too many or too few explanatory variables.)

Your target assets:
{asset_names}.

Your explanation target (AI model result):
{AI_model_result}.

Explanatory variables that AI model considered important (Note: Daily data exists for past 3 months.):
{var_dict}.

Your persona:
{persona}.

Your task:

1. Start interview by introducing yourself.
2. Come up with a challenging and complex first question for the first explanation target.
3. Break this first question into very specific sub-questions. Make {num_subquestions} sub-questions.
  (Caution: From the original question, make sub-questions that can be answered directly from the past daily explanatory variable data. Use exact name for explanatory variables.)
4. Review the answer from the expert and state your overall understanding of previous sub-questions (`## Understanding` header).
   Then, move on to the next explanation target and make new question, saying "Let's move on to the next explanation target!".
  (Note: There are total 3 explanation targets.)
   Also, break the new question into sub-questions (`## Questions` header).
  (Caution: From the original question, make sub-questions that can be answered directly from the past daily explanatory variable data. Use exact name for explanatory variables.)
   Through iteration, boil down to interesting and specific insights.
    - Interesting: Insights that people will find surprising or non-obvious.   
    - Specific: Insights that avoid generalities and include specific figures.
5. To complete the interview, use these criterion:
    - You are very confident about being ready to explain the target result.
    - You have gained enough interesting and specific insights. 
    - There are no more appropriate questions.
   
   Complete the interview like this:
    - Say "I'd like to finish my interview with the expert since I'm ready for impressive explanation!".
    - Briefly state the prepared explanation and how this explanation can be considered an achievement of your goal.

Output format:

1. Include no preamble.
2. Use markdown formatting and follow this structure:

   ## Understanding
   <Overall understanding>
   ## Questions
   1. [Question] <Follow-up question>
   2. [Sub-questions]
      [Sub-question 1] <First sub-question>
      [Sub-question 2] <Second sub-question>
      [Sub-question 3] <Third sub-question>.

Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied."""

def generate_question(state: InterviewState):
    """ Node to generate a question """

    # Get state
    analyst = state["analyst"]
    messages = state["messages"]
    
    # LLM
    system_message = question_instructions.format(
        asset_names=asset_names,
        AI_model_result=AI_model_result,
        var_dict=var_dict,
        persona=analyst.persona,
        num_subquestions=num_subquestions
        )
    question = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    # Write messages to state
    return {"messages": [question]}

#### B. Context 추출 & Answer 생성

In [24]:
from langchain_core.messages import get_buffer_string

# Extract context
extract_instructions = """You will be given a conversation between an analyst and an expert.

Your goal is to help the expert answer the analyst's questions by providing context.

Your target assets:
{asset_names}.

Your task:

1. Analyze the full conversation and pay particular attention to the analyst's last message, which contains sub-questions.
2. Extract important context for each sub-question from table (`## Contexts` header).
   Follow these guidelines:
    - Handle ALL sub-questions posed by the analyst at last turn, one by one.
    - When there is no context to extract, DO NOT make up any context and just return "No context for this sub-question!".
3. Include the source (`### Source` header).

Output format:

1. Include no preamble.
2. Use markdown formatting and follow this structure:

   ## Contexts
   1. [Question] <Follow-up question>
   2. [Sub-questions]
      [Sub-question 1] <First sub-question>: <Context for first sub-question>
      [Sub-question 2] <Second sub-question>: <Context for second sub-question>
      [Sub-question 3] <Third sub-question>: <Context for third sub-question>.
   ### Source
   <Source> or None (e.g., [Table 101]. DO NOT use this format: [1].).

Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied.

"""

# Equity and volatility indicator
extract_101_instructions = extract_instructions + """Available source:

Table 101 has explanatory variables related to equity and volatility indicator.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

# Government bond yield
extract_102_instructions = extract_instructions + """Available source:

Table 102 has explanatory variables related to government bond yield.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

# Commodities and Futures
extract_103_instructions = extract_instructions + """Available source:

Table 103 has explanatory variables related to commodities and futures.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

# Currency exchange rates
extract_104_instructions = extract_instructions + """Available source:

Table 104 has explanatory variables related to currency exchange rates.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

# Market index
extract_105_instructions = extract_instructions + """Available source:

Table 105 has explanatory variables related to market index.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

In [25]:
# DSC101TH
def search_equity_and_volatility(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_101_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_101,
        table_data=table_101_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

# DSC102TH
def search_government_bond_yield(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_102_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_102,
        table_data=table_102_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

# DSC103TH
def search_commodities_and_futures(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_103_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_103,
        table_data=table_103_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

# DSC104TH
def search_currency_exchange_rates(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_104_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_104,
        table_data=table_104_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

# DSC105TH
def search_market_index(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_105_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_105,
        table_data=table_105_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

In [26]:
answer_instructions = """You are an expert being interviewed by an analyst.

Your goal is to answer the analyst's questions based on the provided context.

Your target assets:
{asset_names}.

Your explanation target (AI model result):
{AI_model_result}.

Analyst's persona:
{persona}.

Your task:

1. Analyze the full conversation and pay particular attention to the analyst's last message, which contains sub-questions.
2. Answer the questions (`## Answer` header).
   Follow these guidelines:
    - Answer all sub-questions posed by the analyst at last turn, one by one, using the provided context.
     (Caution: DO NOT handle just single sub-quesiton.)
    - Only use the provided context. When the provided context is insufficient, DO NOT make up any answer and just mention that more context is necessary.
     (Caution: DO NOT leverage external or unknown contexts that are not provided.)
    - Include the source next to any relevant statements (e.g., for Table 102 use [Table 102].).
    - Create a list of sources (`### Sources` header).
    
Output format:

1. Include no preamble.
2. Use markdown formatting and follow this structure:

   ## Answer
   [Sub-question 1] <First sub-question>: <Answer for first sub-question>
   [Sub-question 2] <Second sub-question>: <Answer for second sub-question>
   [Sub-question 3] <Third sub-question>: <Answer for third sub-question>
   ### Sources
   <Source 1>, <Source 2>, etc. (e.g., [Table 101], [Table 102]. DO NOT use this format: [1], [2].).

Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied.

Provided context:
{context}"""

In [27]:
def generate_answer(state: InterviewState):
    """ Node to answer a question """

    # Get state
    analyst = state["analyst"]
    context = state["context"]
    messages = state["messages"]

    # LLM
    system_message = answer_instructions.format(
        asset_names=asset_names,
        AI_model_result=AI_model_result,
        persona=analyst.persona,
        context=context
        )
    answer = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    # Name the message as coming from the expert
    answer.name = "expert"
    
    return {"messages": [answer]}

def save_interview(state: InterviewState):
    """ Save interviews """

    # Get state
    messages = state["messages"]
    
    # Convert interview to a string
    interview = get_buffer_string(messages)
    
    return {"interview": interview}

def route_messages(state: InterviewState, 
                   name: str = "expert"):
    """ Route between question and answer """
    
    # Get state
    messages = state["messages"]
    max_num_turns = state["max_num_turns"]

    # Check the number of expert answers 
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage) and m.name == name]
    )

    # End if expert has answered more than the max turns
    if num_responses >= max_num_turns:
        return 'save_interview'

    # This router is run after each question - answer pair 
    # Get the last question asked to check if it signals the end of discussion
    last_question = messages[-2]
    
    if "I'd like to finish my interview with expert since I'm ready for impressive explanation!" in last_question.content:
        return 'save_interview'
    
    return "ask_question"

#### C. Section 작성

In [28]:
section_writer_instructions = """You are a technical writer specializing in the finance field.

Your goal is to write a section based on the provided interview transcript.

Your task:

1. Review the information of interviewer:
{description}
2. Create a narrative by gathering and understanding valid sub-question and answer pairs from the interview transcript.
3. Write Body part (`## Section` header).
   Follow these guidelines:
    - DO NOT mention any interviewer and interviewee names.
    - Aim for approximately 500 words maximum and 3 paragraphs with sub-title (`### Sub-title` header). Keep the flow of the interview as close as possible.
     (Caution: DO NOT over-abbreviate original meaningful insights and figures.)
    - Preserve all the existing specific figures and sources in the interviewee's answer.
     (Caution: When making a new sentence by combining sentences with explicit source, include the new source next to this made-up sentence. It can be multiple sources like [Table 101], [Table 102].).
4. Write Sources part (`### Sources` header).
   Follow these guidelines:
    - Create a list of sources.
    - Be sure that there is no redundant sources.

Output format:

1. Include no preamble.
2. Use markdown formatting and follow this structure:
    ## Section
    ### Sub-title 1
    ### Sub-title 2
    ### Sub-title 3
    ### Sources (e.g., [Table 101], [Table 102]. DO NOT use this format: [1], [2].).

Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied."""

def write_section(state: InterviewState):
    """ Node to answer a question """

    # Get state
    interview = state["interview"]
    analyst = state["analyst"]
    
    # LLM (use context)
    system_message = section_writer_instructions.format(description=analyst.description)
    section = llm.invoke(
        [SystemMessage(content=system_message)] +
        [HumanMessage(content=f"Use this interview transcript to write your section: {interview}")]
        ) 
    
    # Append it to state
    return {"sections": [section.content]}

In [29]:
# Add nodes and edges 
interview_builder = StateGraph(InterviewState)
interview_builder.add_node("ask_question", generate_question)
interview_builder.add_node("search_equity_and_volatility", search_equity_and_volatility)
interview_builder.add_node("search_government_bond_yield", search_government_bond_yield)
interview_builder.add_node("search_commodities_and_futures", search_commodities_and_futures)
interview_builder.add_node("search_currency_exchange_rates", search_currency_exchange_rates)
interview_builder.add_node("search_market_index", search_market_index)
interview_builder.add_node("answer_question", generate_answer)
interview_builder.add_node("save_interview", save_interview)
interview_builder.add_node("write_section", write_section)

# Flow
interview_builder.add_edge(START, "ask_question")
interview_builder.add_edge("ask_question", "search_equity_and_volatility")
interview_builder.add_edge("ask_question", "search_government_bond_yield")
interview_builder.add_edge("ask_question", "search_commodities_and_futures")
interview_builder.add_edge("ask_question", "search_currency_exchange_rates")
interview_builder.add_edge("ask_question", "search_market_index")
interview_builder.add_edge("search_equity_and_volatility", "answer_question")
interview_builder.add_edge("search_government_bond_yield", "answer_question")
interview_builder.add_edge("search_commodities_and_futures", "answer_question")
interview_builder.add_edge("search_currency_exchange_rates", "answer_question")
interview_builder.add_edge("search_market_index", "answer_question")
interview_builder.add_conditional_edges("answer_question", route_messages, ['ask_question', 'save_interview'])
interview_builder.add_edge("save_interview", "write_section")
interview_builder.add_edge("write_section", END)

# Interview 
memory = MemorySaver()
interview_graph = interview_builder.compile(checkpointer=memory).with_config(run_name="Conduct Interviews")

### 3-3. Interview 수행 (Parallelization)

In [30]:
from langgraph.constants import Send

# Conditional edge
def initiate_all_interviews(state: ResearchGraphState):
    """ This is the "map" step where we run each interview sub-graph using Send API """    

    # Check if human feedback
    human_analyst_feedback = state.get('human_analyst_feedback')
    if human_analyst_feedback:
        # Return to create_analysts
        return "create_analysts"

    # Otherwise kick off interviews in parallel via Send() API
    else:
        messages = [HumanMessage(content="So you said you were explaining the AI model result of portfolio optimization?")]
        return [Send("conduct_interview",
                     {"analyst": analyst,
                      "max_num_turns": max_num_turns,
                      "messages": messages}) for analyst in state["analysts"]]

### 3-4. Interpretation

In [31]:
interpreter_instructions = """You are a certified investment manager with extensive finance field experience.

Your goal is to interpret the AI model result of portfolio optimization from your professional perspective and data.

It is extremely important to focus on on high-level themes such as market dynamics, fund flows, and potential risks.

Your target assets:
{asset_names}.

Your explanation target (AI model result):
{AI_model_result}.

Your task:

1. Analyze all the sections written from the interview result.
   A team of analysts conducted interview. Each analyst has done two things: 
    - They conducted an interview with an expert.
    - They wrote up their finding into a section.
2. Think carefully about the provided sections.
3. Consolidate the insights from all the sections to come up with high-level insights.
4. Create a holistic narrative that reflects your persona and connects the insights together.
   This holistic narrative focuses on high-level themes such as market dynamics, fund flows, and potential risks.
5. Write Body part (`## Interpretation` header).
   Follow these guidelines:
    - Focus on the provided sections. When necessary, you can additionally leverage your own knowledge.
    - Emphasize the novelty of your insights.
    - DO NOT mention any imaginary names.
    - Aim for approximately 750 words maximum and 3 paragraphs with sub-title (`### Sub-title` header). Each paragraph should correspond to individual explanation target.
     (Caution: Avoid vague phrases that are not specific and have no impact. DO NOT over-abbreviate original meaningful insights and figures.)
    - Preserve all the existing specific figures and sources in the provided sections.
     (Caution: When making a new sentence by combining sentences with explicit sources, include the new source next to this made-up sentence. It can be multiple sources like [Table 101], [Table 102].).
6. Write Sources part (`### Sources` header).
   Follow these guidelines:
    - Create a list of sources.
    - Be sure that there is no redundant sources.
7. Come up with an engaging and focused title (`# Title` header).
  (Caution: Avoid vague phrases that are not specific and have no impact.)
  
Output format:

1. Include no preamble.
2. Use markdown formatting and follow this structure:
    # Title
    ## Interpretation
    ### Sub-title 1
    ### Sub-title 2
    ### Sub-title 3
    ### Sources.

Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied.

"""

interpreter_risk_seeking_instructions = interpreter_instructions + """Your persona:

1. You are a certified investment manager and need to explain the AI model result of portfolio optimization to your customers.
2. Your customors have risk-seeking investment style: risk-seeking level is 5/5."""

interpreter_stability_seeking_instructions = interpreter_instructions + """Your persona:

1. You are a certified investment manager and need to explain the AI model result of portfolio optimization to your customers.
2. Your customors have stability-seeking investment style: risk-seeking level is 4/5."""

interpreter_balanced_instructions = interpreter_instructions + """Your persona:

1. You are a certified investment manager and need to explain the AI model result of portfolio optimization to your customers.
2. Your customors have balanced investment style: risk-seeking level is 3/5."""

In [32]:
def interpret_risk_seeking(state: ResearchGraphState):
    # Full set of sections
    sections = state["sections"]

    # Concat all sections together
    formatted_str_sections = "\n\n".join([f"{section}" for section in sections])
    
    # LLM
    system_message = interpreter_risk_seeking_instructions.format(
        asset_names=asset_names,
        AI_model_result=AI_model_result)    
    interpretation = llm.invoke(
        [SystemMessage(content=system_message)] + 
        [HumanMessage(content=f"Write interpretation based on the written sections. Here are the sections: {formatted_str_sections}")])
    
    return {"interpretations": [interpretation.content]}

def interpret_stability_seeking(state: ResearchGraphState):
    # Full set of sections
    sections = state["sections"]

    # Concat all sections together
    formatted_str_sections = "\n\n".join([f"{section}" for section in sections])
    
    # LLM
    system_message = interpreter_stability_seeking_instructions.format(
        asset_names=asset_names,
        AI_model_result=AI_model_result)    
    interpretation = llm.invoke(
        [SystemMessage(content=system_message)] + 
        [HumanMessage(content=f"Write interpretation based on the written sections. Here are the sections: {formatted_str_sections}")])
    
    return {"interpretations": [interpretation.content]}

def interpret_balanced(state: ResearchGraphState):
    # Full set of sections
    sections = state["sections"]

    # Concat all sections together
    formatted_str_sections = "\n\n".join([f"{section}" for section in sections])
    
    # LLM
    system_message = interpreter_balanced_instructions.format(
        asset_names=asset_names,
        AI_model_result=AI_model_result)    
    interpretation = llm.invoke(
        [SystemMessage(content=system_message)] + 
        [HumanMessage(content=f"Write interpretation based on the written sections. Here are the sections: {formatted_str_sections}")])
    
    return {"interpretations": [interpretation.content]}    

### 3-5. Report 작성

In [33]:
report_writer_instructions = """You are a technical writer specializing in the finance field.

Your goal is to write a technical report that explains the AI model result of portfolio optimization by integrating the provided interpretations.

It is extremely important to focus on on high-level themes such as market dynamics, fund flows, and potential risks.

Your target assets:
{asset_names}.

Your explanation target (AI model result):
{AI_model_result}.

Your task:

1. Analyze all the interpretations provided by several certified investment managers with diverse investment style.
2. Think carefully about the provided interpretations.
3. Consolidate the insights from all the interpretations to come up with impressive and powerful conclusion.
4. Create a holistic narrative that reflects your persona and connects the insights together.
   This holistic narrative focuses on high-level themes such as market dynamics, fund flows, and potential risks.
5. Write Body part (`## Analysis` header).
   Follow these guidelines:
    - Assume readers with profound knowledge and diverse investment style.
    - Tone and manner must be technical and clear.
    - Begin by briefly summarizing 3 explanation targets (`### Target` header).
    - Emphasize the novelty of your insights.
    - DO NOT mention any imaginary names.
    - Aim for approximately 750 words maximum and 3 paragraphs with sub-title (`### Sub-title` header). Each paragraph should correspond to individual explanation target.
     (Caution: Avoid vague phrases that are not specific and have no impact. DO NOT over-abbreviate original meaningful insights and figures.)
    - Preserve all the existing specific figures and sources in the provided interpretations.
     (Caution: When making a new sentence by combining sentences with explicit sources, include the new source next to this made-up sentence. It can be multiple sources like [Table 101], [Table 102].).
6. Write Sources part (`### Sources` header).
   Follow these guidelines:
    - Create a list of sources.
    - Be sure that there is no redundant sources.
7. Come up with an engaging and focused title (`# Title` header).
  (Caution: Avoid vague phrases that are not specific and have no impact.)
8. Provide the original English version AND Korean version report.

Output format:

1. Include no preamble.
2. For English version, use markdown formatting and follow this structure:
    # Title
    ## Analysis
    ### Target
    ### Sub-title 1
    ### Sub-title 2
    ### Sub-title 3
    ### Sources.
3. 한국어 버전에 마크다운 포맷을 적용하고 다음 구조를 따르세요:
    # 제목
    ## 분석
    ### 타겟
    ### 소제목 1
    ### 소제목 2
    ### 소제목 3
    ### 출처.
    
Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied."""

In [34]:
def write_report(state: ResearchGraphState):
    # Full set of interpretations
    interpretations = state["interpretations"]
    
    # Concat all interpretations together
    formatted_str_interpretations = "\n\n".join([f"{interpretation}" for interpretation in interpretations])
    
    # LLM
    system_message = report_writer_instructions.format(
        asset_names=asset_names,
        AI_model_result=AI_model_result)    
    report = llm.invoke(
        [SystemMessage(content=system_message)] + 
        [HumanMessage(content=f"Write a report based on the diverse interpretations. Here are the interpretations: {formatted_str_interpretations}")])
    
    return {"content": report.content}

In [35]:
# Add nodes and edges 
builder = StateGraph(ResearchGraphState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)
builder.add_node("conduct_interview", interview_builder.compile())
builder.add_node("interpret_risk_seeking", interpret_risk_seeking)
builder.add_node("interpret_stability_seeking", interpret_stability_seeking)
builder.add_node("interpret_balanced", interpret_balanced)
builder.add_node("write_report", write_report)

# Logic
builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", initiate_all_interviews, ["create_analysts", "conduct_interview"])
builder.add_edge("conduct_interview", "interpret_risk_seeking")
builder.add_edge("conduct_interview", "interpret_stability_seeking")
builder.add_edge("conduct_interview", "interpret_balanced")
builder.add_edge("interpret_risk_seeking", "write_report")
builder.add_edge("interpret_stability_seeking", "write_report")
builder.add_edge("interpret_balanced", "write_report")
builder.add_edge("write_report", END)

# Compile
memory = MemorySaver()
graph = builder.compile(interrupt_before=['human_feedback'], checkpointer=memory)

### 3-6. 실행

In [36]:
# Input
thread = {"configurable": {"thread_id": "1"}}

In [37]:
# Run the graph until the first interruption
for event in graph.stream({"analysts": analysts}, # {"max_analysts": max_analysts}
                          thread, 
                          stream_mode="values"):
    
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 100) 

Name: Emma Thompson
Affiliation: International Financial Advisory Group
Role: Global Market Strategist
Description: Emma focuses on global market dynamics, analyzing how equity and volatility indicators interact with government bond yields to influence portfolio optimization. She is particularly interested in how shifts in major indices like the DAX and FTSE 100, combined with bond yield movements, can signal broader economic trends and impact asset allocation strategies.
----------------------------------------------------------------------------------------------------
Name: Liam Chen
Affiliation: Global Commodities Research Institute
Role: Commodities Analyst
Description: Liam specializes in the commodities and futures markets, examining how fluctuations in key commodities like gold, copper, and crude oil affect portfolio performance. He explores the interplay between commodity prices and equity markets, assessing how these relationships can inform risk management and diversificatio

In [38]:
# Confirm we are happy
further_feedack = None
graph.update_state(
    thread,
    {"human_analyst_feedback": further_feedack},
    as_node="human_feedback"
    )

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1efb946a-b534-6f3e-8002-b9f542f78b65'}}

In [39]:
# Continue
for event in graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)

--Node--
conduct_interview
--Node--
conduct_interview
--Node--
conduct_interview
--Node--
conduct_interview
--Node--
conduct_interview
--Node--
interpret_stability_seeking
--Node--
interpret_risk_seeking
--Node--
interpret_balanced
--Node--
write_report


In [40]:
from IPython.display import Markdown

final_state = graph.get_state(thread)
report = final_state.values.get('content')
Markdown(report)

# Portfolio Reallocation and Market Dynamics: November 2024

## Analysis

### Target

The AI model results indicate three key observations regarding portfolio optimization for the target assets: a significant drop in asset group weights on November 20, 2024, a consistent increase in asset group weights prior to this date, and the unique behavior of AA-ETF_EQUITY-SUSL on the same date. These observations reflect underlying market dynamics, fund flows, and potential risks that influenced investment strategies during this period.

### Market Volatility and Asset Reallocation

The significant drop in asset group weights on November 20, 2024, is a strategic response to heightened market volatility and shifting investor sentiment. The increase in the CBOE SPX Volatility VIX (NEW) index from 15.58 to 17.16 during this period indicates a rise in market uncertainty, prompting investors to reassess their risk exposure [Table 101]. This volatility, coupled with a slight decrease in the Refinitiv United States Government Benchmark Bid Yield 10 Years from 4.416 to 4.388, suggests a shift towards safer investments, as investors sought refuge in lower-risk assets amidst the volatile environment [Table 102]. Additionally, fluctuations in commodity markets, such as the increase in the CBOE GOLD VOLATILITY INDEX from 16.97 to 18.34, further contributed to the reallocation of assets, as investors adjusted their portfolios to mitigate potential risks [Table 101]. These market dynamics collectively underscore the importance of maintaining a flexible investment strategy that can adapt to evolving conditions, particularly in times of increased volatility.

### Factors Driving Asset Weight Increases

Prior to the significant drop on November 20, 2024, asset group weights had been on a consistent upward trajectory, driven by favorable market conditions and positive investor sentiment. The robust performance of major equity indices, such as the CME-Standard and Poors 500 Index Composite CS02, which rose from 5608.25 to 5916.98, reflects a bullish trend in the equity market, encouraging increased exposure to equities [Table 101]. Additionally, the downward trend in the Refinitiv China Government Benchmark Bid Yield 10 Years from 2.183 to 2.091 suggests a shift away from bonds towards equities, as investors capitalized on the favorable conditions for equity investments [Table 102]. The appreciation of the Brazilian Real against the US Dollar, with the exchange rate moving from 5.4785 to 5.77275, also influenced investment strategies, as investors sought to benefit from currency movements [Table 104]. These factors collectively contributed to the increase in asset group weights, as investors sought to capitalize on market opportunities and optimize their portfolios.

### Unique Behavior of AA-ETF_EQUITY-SUSL

The unique behavior of AA-ETF_EQUITY-SUSL on November 20, 2024, where it increased in weight while other asset groups experienced a decline, can be attributed to specific market dynamics and currency fluctuations. The strengthening of the United States Dollar against the Euro, with the exchange rate declining from 1.11305 to 1.0544, made US-based assets more attractive to international investors, potentially explaining the increased weight of AA-ETF_EQUITY-SUSL [Table 104]. Additionally, the slight decrease in the Refinitiv United States Government Benchmark Bid Yield 3 Years from 4.277 to 4.237 may have influenced investor decisions, as they sought to adjust their portfolios in response to changing interest rate expectations [Table 102]. These factors highlight the importance of considering currency movements and interest rate trends when making investment decisions, as they can significantly impact the relative attractiveness of different asset classes and influence portfolio allocations.

### Sources

[Table 101], [Table 102], [Table 103], [Table 104].

# 포트폴리오 재배치 및 시장 역학: 2024년 11월

## 분석

### 타겟

AI 모델 결과는 대상 자산에 대한 포트폴리오 최적화와 관련하여 세 가지 주요 관찰을 나타냅니다: 2024년 11월 20일에 자산 그룹 가중치의 상당한 하락, 이 날짜 이전의 자산 그룹 가중치의 일관된 증가, 그리고 같은 날짜에 AA-ETF_EQUITY-SUSL의 독특한 행동입니다. 이러한 관찰은 이 기간 동안 투자 전략에 영향을 미친 기본적인 시장 역학, 자금 흐름 및 잠재적 위험을 반영합니다.

### 시장 변동성과 자산 재배치

2024년 11월 20일에 자산 그룹 가중치의 상당한 하락은 시장 변동성 증가와 투자자 심리 변화에 대한 전략적 대응입니다. 이 기간 동안 CBOE SPX Volatility VIX (NEW) 지수가 15.58에서 17.16으로 증가한 것은 시장 불확실성이 증가했음을 나타내며, 이는 투자자들이 위험 노출을 재평가하도록 촉발했습니다 [Table 101]. 이러한 변동성과 함께 Refinitiv 미국 정부 벤치마크 입찰 수익률 10년물이 4.416에서 4.388로 약간 감소한 것은 투자자들이 변동성이 큰 환경 속에서 더 안전한 자산으로 피신하려는 움직임을 시사합니다 [Table 102]. 또한, CBOE GOLD VOLATILITY INDEX가 16.97에서 18.34로 증가하는 등 상품 시장의 변동성은 투자자들이 잠재적 위험을 완화하기 위해 포트폴리오를 조정함에 따라 자산 재배치에 기여했습니다 [Table 101]. 이러한 시장 역학은 특히 변동성이 증가하는 시기에 진화하는 조건에 적응할 수 있는 유연한 투자 전략을 유지하는 것의 중요성을 강조합니다.

### 자산 가중치 증가를 이끄는 요인

2024년 11월 20일의 상당한 하락 이전에 자산 그룹 가중치는 유리한 시장 조건과 긍정적인 투자자 심리에 의해 일관되게 증가했습니다. CME-Standard and Poors 500 Index Composite CS02와 같은 주요 주식 지수의 강력한 성과는 주식 시장의 강세 추세를 반영하며, 주식에 대한 노출을 증가시키도록 장려했습니다 [Table 101]. 또한, Refinitiv 중국 정부 벤치마크 입찰 수익률 10년물이 2.183에서 2.091로 하락하는 추세는 투자자들이 주식 투자에 유리한 조건을 활용함에 따라 채권에서 주식으로의 이동을 시사합니다 [Table 102]. 브라질 레알이 미국 달러 대비 5.4785에서 5.77275로 평가 절상된 것도 투자 전략에 영향을 미쳤으며, 투자자들이 통화 움직임에서 이익을 얻으려 했습니다 [Table 104]. 이러한 요인들은 투자자들이 시장 기회를 활용하고 포트폴리오를 최적화하려고 함에 따라 자산 그룹 가중치 증가에 기여했습니다.

### AA-ETF_EQUITY-SUSL의 독특한 행동

2024년 11월 20일에 다른 자산 그룹이 하락을 경험한 반면 AA-ETF_EQUITY-SUSL의 가중치가 증가한 독특한 행동은 특정 시장 역학과 통화 변동에 기인할 수 있습니다. 미국 달러가 유로 대비 강세를 보이며 환율이 1.11305에서 1.0544로 하락한 것은 국제 투자자들에게 미국 기반 자산을 더 매력적으로 만들었을 가능성이 있으며, 이는 AA-ETF_EQUITY-SUSL의 가중치 증가를 설명할 수 있습니다 [Table 104]. 또한, Refinitiv 미국 정부 벤치마크 입찰 수익률 3년물이 4.277에서 4.237로 약간 감소한 것은 투자자들이 금리 기대 변화에 대응하여 포트폴리오를 조정하려 했음을 시사할 수 있습니다 [Table 102]. 이러한 요인은 투자 결정을 내릴 때 통화 움직임과 금리 추세를 고려하는 것의 중요성을 강조하며, 이는 다양한 자산 클래스의 상대적 매력에 크게 영향을 미치고 포트폴리오 할당에 영향을 미칠 수 있습니다.

### 출처

[Table 101], [Table 102], [Table 103], [Table 104].

In [41]:
report

'# Portfolio Reallocation and Market Dynamics: November 2024\n\n## Analysis\n\n### Target\n\nThe AI model results indicate three key observations regarding portfolio optimization for the target assets: a significant drop in asset group weights on November 20, 2024, a consistent increase in asset group weights prior to this date, and the unique behavior of AA-ETF_EQUITY-SUSL on the same date. These observations reflect underlying market dynamics, fund flows, and potential risks that influenced investment strategies during this period.\n\n### Market Volatility and Asset Reallocation\n\nThe significant drop in asset group weights on November 20, 2024, is a strategic response to heightened market volatility and shifting investor sentiment. The increase in the CBOE SPX Volatility VIX (NEW) index from 15.58 to 17.16 during this period indicates a rise in market uncertainty, prompting investors to reassess their risk exposure [Table 101]. This volatility, coupled with a slight decrease in the